In [1]:
# STARTING THE CODE 
# -----------------------------------------------------

# Pour que les graphs s'affichent tous seuls dans faire de commande show (selo Pierre)
#%matplotlib inline
# Importing seaborn package for graphics
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

from matplotlib.ticker import ScalarFormatter
from matplotlib.lines import Line2D

# Importing seaborn package for graphics
import seaborn as sns

# Importing pandas package for data
import pandas as pd

# Importing other packages
import numpy as np
import math

# Importing to perform a linear regression
from sklearn.linear_model import LinearRegression

# Import functools to merge several dataframes
from functools import reduce

from sklearn.preprocessing import MinMaxScaler

# Apllying the default theme to seaborn
#sns.set_theme()


In [2]:
import os
print(os.getcwd())
os.chdir('/Users/crosnier')

/Users/crosnier/Desktop/PDM


In [7]:
# IMPORTING NECESSARY DATA

# LCA database
LCA_database = pd.read_excel('Desktop/PDM/Modelling_data/LCA impacts/LCA_database_ecoinvent_agribalyse.xlsx', sheet_name='scoped_env_impacts', header=0)

# Livestocks matched LCA
livestock_matched_LCA = pd.read_excel('Desktop/PDM/Modelling_data/computed_data/animals_matched_LCA.xlsx', sheet_name='livestock_matched_2050', header=0)

# Data from the scenarios
crops_fish = pd.read_excel('Desktop/PDM/Modelling_data/Environmental impacts/calculs_impacts_scenarios_31-10-2024.xlsx', sheet_name='crops_fish', header=1)
livestock = pd.read_excel('Desktop/PDM/Modelling_data/Environmental impacts/calculs_impacts_scenarios_31-10-2024.xlsx', sheet_name='livestock_conv_org_for_code', header=0)
#feed = pd.read_excel('Desktop/PDM/Modelling_data/Environmental impacts/calculs_impacts_scenarios.xlsx', sheet_name='feed', header=0)
#crops_fish_T = pd.read_excel('Desktop/PDM/Modelling_data/Environmental impacts/calculs_impacts_scenarios.xlsx', sheet_name='crops_fish_T', header=0)
#livestock_T = pd.read_excel('Desktop/PDM/Modelling_data/Environmental impacts/calculs_impacts_scenarios.xlsx', sheet_name='livestock_T', header=0)
#livestock_T2 = pd.read_excel('Desktop/PDM/Modelling_data/Environmental impacts/calculs_impacts_scenarios.xlsx', sheet_name='livestock_T2', header=0)
#crop_area_T = pd.read_excel('Desktop/PDM/Modelling_data/Environmental impacts/calculs_impacts_scenarios.xlsx', sheet_name='crop_area_T', header=0)
crop_fish_no_feed = pd.read_excel('Desktop/PDM/Modelling_data/Environmental impacts/calculs_impacts_scenarios_31-10-2024.xlsx', sheet_name='crop_fish_food_without_feed', header=0)
#livestock_conv_org_T = pd.read_excel('Desktop/PDM/Modelling_data/Environmental impacts/calculs_impacts_scenarios.xlsx', sheet_name='livestock_conv_org_T', header=0)

In [8]:
# ENVIRONMENTAL IMPACTS: PREPARATION

# Importation of LCA item name to match the LCA database (food_balance_items)
fb_matched_LCA_items = pd.read_excel('Desktop/PDM/Modelling_data/Environmental impacts/food_balance_items.xlsx', sheet_name='plant_prod_CH', header=0)

# Creating separated df for each category: 'Conventionnal CH', 'Conventionnal GLO', 'Organic CH', 'Swiss integrated production'
conv_ch = fb_matched_LCA_items.copy()                    # Creating copies
org_ch = fb_matched_LCA_items.copy()

# Selecting the columns based on the desired df
conv_ch = conv_ch[['Item', 'Conventionnal CH']]          
org_ch = org_ch[['Item', 'Organic CH']]

org_ch

,Item,Organic CH
0,Apples and products,"Apple, organic, national average, at orchard/kg"
1,Barley and products,"Barley grain, organic {CH}| barley production,..."
2,"Cereals, Other","Barley grain, organic {CH}| barley production,..."
3,"Fruits, other","Pear, organic, at farm gate/kg"
4,Grapes and products (excl wine),"Grape, organic, variety mix, Languedoc-Roussil..."
5,Maize and products,"Maize grain, organic {CH}| production | Cut-of..."
6,Millet and products,"Millet, organic - Computed"
7,Nuts and products,"Dried inshell walnut, traditional varieties, o..."
8,Oats,"Oat, organic - Computed"
9,Onions,"Onion, organic - Computed"


In [9]:
# IMPACTS COMPUTATION - BAU SWISS

# Preparation CROP & FISH - FEED NOT CONSIDERED
crops_fish_BAU_swiss = crop_fish_no_feed.copy()
columns_to_drop = ['Mass BAU Swiss [kt]', 'Mass org BAU Lancet [kt]', 'Mass conv BAU Lancet [kt]', 'Mass OA Swiss [kt]', 'Mass OA Lancet [kt]', 'Mass BAU Lancet [kt]',
                  'Mass org ConvBASE [kt]', 'Mass conv ConvBASE [kt]', 'Mass ConvBASE [kt]', 'Mass OrgBASE [kt]']
crops_fish_BAU_swiss  = crops_fish_BAU_swiss.drop(columns_to_drop, axis=1)
# Dropping the row with Citrus, other
crops_fish_BAU_swiss = crops_fish_BAU_swiss.drop(crops_fish_BAU_swiss[crops_fish_BAU_swiss['Item'] == 'Citrus, Other'].index)

# Preparation LIVESTOCK
livestock_BAU_swiss = livestock.copy()
columns_to_drop = ['BAU Lancet', 'OA Swiss', 'OA Lancet', 'ConvBASE', 'OrgBASE']
livestock_BAU_swiss  = livestock_BAU_swiss.drop(columns_to_drop, axis=1)

# CROP & FISH ========================================
# CONVENTIONNAL AGR ___________________________
# Merging the items with conv_ch
crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss.merge(conv_ch, on='Item', how='left') 
# Merging with the LCA database
crops_fish_BAU_swiss_conv = pd.merge(crops_fish_BAU_swiss_conv, LCA_database, how='left', left_on='Conventionnal CH', right_on='Item')  
# Dropping columns
columns_to_drop = ['Mass org BAU Swiss [kt]', 'Conventionnal CH', 'Item_y']
crops_fish_BAU_swiss_conv  = crops_fish_BAU_swiss_conv.drop(columns_to_drop, axis=1)
# Renaming columns
crops_fish_BAU_swiss_conv.rename(columns = {'Item_x':'Item'}, inplace = True)
# Computing the environmental impacts
target_column_index = 1  # Index of the target column to multiply other columns with
columns_to_multiply = range(2, 11)  # Range of column indices to multiply
target_column = crops_fish_BAU_swiss_conv.iloc[:, target_column_index] * 10**6  # Converting from kt to kg
crops_fish_BAU_swiss_conv.iloc[:, columns_to_multiply] = crops_fish_BAU_swiss_conv.iloc[:, columns_to_multiply].multiply(target_column, axis=0)

# ORGANIC AGR _________________________________
# Merging the items with conv_ch
crops_fish_BAU_swiss_org = crops_fish_BAU_swiss.merge(org_ch, on='Item', how='left') 
# Merging with the LCA database
crops_fish_BAU_swiss_org = pd.merge(crops_fish_BAU_swiss_org, LCA_database, how='left', left_on='Organic CH', right_on='Item')  
# Dropping columns
columns_to_drop = ['Mass conv BAU Swiss [kt]', 'Organic CH', 'Item_y']
crops_fish_BAU_swiss_org  = crops_fish_BAU_swiss_org.drop(columns_to_drop, axis=1)
# Renaming columns
crops_fish_BAU_swiss_org.rename(columns = {'Item_x':'Item'}, inplace = True)
# Computing the environmental impacts
target_column_index = 1  # Index of the target column to multiply other columns with
columns_to_multiply = range(2, 11)  # Range of column indices to multiply
target_column = crops_fish_BAU_swiss_org.iloc[:, target_column_index] * 10**6  # Converting from kt to kg
crops_fish_BAU_swiss_org.iloc[:, columns_to_multiply] = crops_fish_BAU_swiss_org.iloc[:, columns_to_multiply].multiply(target_column, axis=0)

# ORG & CONV _________________________________
# Filling the NaN with zeros
crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss_conv.fillna(0)
crops_fish_BAU_swiss_org = crops_fish_BAU_swiss_org.fillna(0)
# Setting Item as index
crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss_conv.set_index('Item')
crops_fish_BAU_swiss_org = crops_fish_BAU_swiss_org.set_index('Item')
#Dropping the mass columns
columns_to_drop = ['Mass org BAU Swiss [kt]']
crops_fish_BAU_swiss_org = crops_fish_BAU_swiss_org.drop(columns_to_drop, axis=1)
columns_to_drop = ['Mass conv BAU Swiss [kt]']
crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss_conv.drop(columns_to_drop, axis=1)
# Summing the df
crops_fish_BAU_swiss = crops_fish_BAU_swiss_conv.add(crops_fish_BAU_swiss_org)

# LIVESTOCK ===================================
# Merging the items with conv_ch
livestock_BAU_swiss = livestock_BAU_swiss.merge(livestock_matched_LCA, on='Item', how='inner') 
# Merging with the LCA database
livestock_BAU_swiss = pd.merge(livestock_BAU_swiss, LCA_database, how='inner', left_on='Item LCA', right_on='Item') 
# Dropping columns
columns_to_drop = ['Lifetime LCA [y]', 'Item LCA', 'Item_y']
livestock_BAU_swiss = livestock_BAU_swiss.drop(columns_to_drop, axis=1)
# Renaming columns
livestock_BAU_swiss.rename(columns = {'Item_x':'Item'}, inplace = True)
# Computing the environmental impacts
target_column_index = 2  # Index of the target column to multiply other columns with
columns_to_multiply = range(3, 12)  # Range of column indices to multiply
target_column = livestock_BAU_swiss.iloc[:, target_column_index]
livestock_BAU_swiss.iloc[:, columns_to_multiply] = livestock_BAU_swiss.iloc[:, columns_to_multiply].multiply(target_column, axis=0)
# Diving by the lifetime
target_column_index = 1  # Index of the target column to divide other columns with => 'Lifetime [y]'
columns_to_multiply = range(3, 12)  # Range of column indices to multiply
target_column = livestock_BAU_swiss.iloc[:, target_column_index]
livestock_BAU_swiss.iloc[:, columns_to_multiply] = livestock_BAU_swiss.iloc[:, columns_to_multiply].divide(target_column, axis=0)
# Dropping the lifetspan and nb of animals
columns_to_drop = ['Lifespan [yr]', 'BAU Swiss']
livestock_BAU_swiss = livestock_BAU_swiss.drop(columns_to_drop, axis=1)
# Separating the df conv vs org
BAU_org = livestock_BAU_swiss[livestock_BAU_swiss['Item'].str.endswith('Org')]
BAU_conv = livestock_BAU_swiss[livestock_BAU_swiss['Item'].str.endswith('Conv')]
# Creating copies
BAU_org = BAU_org.copy()
BAU_conv = BAU_conv.copy()
# Drop '- Org' and '- Conv'
BAU_org.loc[:, 'Item'] = BAU_org['Item'].str.replace(' - Org', '')
BAU_conv.loc[:, 'Item'] = BAU_conv['Item'].str.replace(' - Conv', '')
# Setting Item as index
BAU_org = BAU_org.set_index('Item')
BAU_conv = BAU_conv.set_index('Item')
# Summing the df
livestock_BAU_swiss = BAU_conv.add(BAU_org)

# CROP, FISH & LIVESTOCK ===================================
BAU_swiss = pd.concat([crops_fish_BAU_swiss, livestock_BAU_swiss])

# EXPORTING TO EXCEL
BAU_swiss.to_excel(r'Desktop/PDM/Modelling_data/Environmental impacts/scenarios_from_code_31-10-2024.xlsx', sheet_name='BAU_swiss', index=True) 

BAU_swiss.sum()

Climate change [ kg CO2 eq]                2.487856e+09
Acidification [ mol H+ eq]                 4.041519e+07
Eutrophication, freshwater [ kg P eq]      4.078610e+05
Eutrophication, marine [ kg N eq]          1.977779e+07
Eutrophication, terrestrial [ mol N eq]    1.760727e+08
Ecotoxicity, freshwater [ CTUe]            7.874238e+10
Land use [ Pt]                             2.239868e+11
Water use [ m3 depriv,]                    3.685004e+09
Resource use, fossils [ MJ]                1.179008e+10
dtype: float64

In [10]:
# IMPACTS COMPUTATION - BAU Lancet

# Preparation CROP & FISH
crops_fish_BAU_swiss = crop_fish_no_feed.copy()
columns_to_drop = ['Mass BAU Swiss [kt]', 'Mass org BAU Swiss [kt]', 'Mass conv BAU Swiss [kt]', 'Mass OA Swiss [kt]', 'Mass OA Lancet [kt]', 'Mass BAU Lancet [kt]',
                  'Mass org ConvBASE [kt]', 'Mass conv ConvBASE [kt]', 'Mass ConvBASE [kt]', 'Mass OrgBASE [kt]']
crops_fish_BAU_swiss  = crops_fish_BAU_swiss.drop(columns_to_drop, axis=1)
# Dropping the row with Citrus, other
crops_fish_BAU_swiss = crops_fish_BAU_swiss.drop(crops_fish_BAU_swiss[crops_fish_BAU_swiss['Item'] == 'Citrus, Other'].index)

# Preparation LIVESTOCK
livestock_BAU_swiss = livestock.copy()
columns_to_drop = ['BAU Swiss', 'OA Lancet', 'OA Swiss', 'ConvBASE', 'OrgBASE']
livestock_BAU_swiss  = livestock_BAU_swiss.drop(columns_to_drop, axis=1)

# CROP & FISH ========================================
# CONVENTIONNAL AGR ___________________________
# Merging the items with conv_ch
crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss.merge(conv_ch, on='Item', how='left') 
# Merging with the LCA database
crops_fish_BAU_swiss_conv = pd.merge(crops_fish_BAU_swiss_conv, LCA_database, how='left', left_on='Conventionnal CH', right_on='Item')  
# Dropping columns
columns_to_drop = ['Mass org BAU Lancet [kt]', 'Conventionnal CH', 'Item_y']
crops_fish_BAU_swiss_conv  = crops_fish_BAU_swiss_conv.drop(columns_to_drop, axis=1)
# Renaming columns
crops_fish_BAU_swiss_conv.rename(columns = {'Item_x':'Item'}, inplace = True)
# Computing the environmental impacts
target_column_index = 1  # Index of the target column to multiply other columns with
columns_to_multiply = range(2, 11)  # Range of column indices to multiply
target_column = crops_fish_BAU_swiss_conv.iloc[:, target_column_index] * 10**6  # Converting from kt to kg
crops_fish_BAU_swiss_conv.iloc[:, columns_to_multiply] = crops_fish_BAU_swiss_conv.iloc[:, columns_to_multiply].multiply(target_column, axis=0)

# ORGANIC AGR _________________________________
# Merging the items with conv_ch
crops_fish_BAU_swiss_org = crops_fish_BAU_swiss.merge(org_ch, on='Item', how='left') 
# Merging with the LCA database
crops_fish_BAU_swiss_org = pd.merge(crops_fish_BAU_swiss_org, LCA_database, how='left', left_on='Organic CH', right_on='Item')  
# Dropping columns
columns_to_drop = ['Mass conv BAU Lancet [kt]', 'Organic CH', 'Item_y']
crops_fish_BAU_swiss_org  = crops_fish_BAU_swiss_org.drop(columns_to_drop, axis=1)
# Renaming columns
crops_fish_BAU_swiss_org.rename(columns = {'Item_x':'Item'}, inplace = True)
# Computing the environmental impacts
target_column_index = 1  # Index of the target column to multiply other columns with
columns_to_multiply = range(2, 11)  # Range of column indices to multiply
target_column = crops_fish_BAU_swiss_org.iloc[:, target_column_index] * 10**6  # Converting from kt to kg
crops_fish_BAU_swiss_org.iloc[:, columns_to_multiply] = crops_fish_BAU_swiss_org.iloc[:, columns_to_multiply].multiply(target_column, axis=0)

# ORG & CONV _________________________________
# Filling the NaN with zeros
crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss_conv.fillna(0)
crops_fish_BAU_swiss_org = crops_fish_BAU_swiss_org.fillna(0)
# Setting Item as index
crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss_conv.set_index('Item')
crops_fish_BAU_swiss_org = crops_fish_BAU_swiss_org.set_index('Item')
#Dropping the mass columns
columns_to_drop = ['Mass org BAU Lancet [kt]']
crops_fish_BAU_swiss_org = crops_fish_BAU_swiss_org.drop(columns_to_drop, axis=1)
columns_to_drop = ['Mass conv BAU Lancet [kt]']
crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss_conv.drop(columns_to_drop, axis=1)
# Summing the df
crops_fish_BAU_swiss = crops_fish_BAU_swiss_conv.add(crops_fish_BAU_swiss_org)

# LIVESTOCK ===================================
# Merging the items with conv_ch
livestock_BAU_swiss = livestock_BAU_swiss.merge(livestock_matched_LCA, on='Item', how='inner') 
# Merging with the LCA database
livestock_BAU_swiss = pd.merge(livestock_BAU_swiss, LCA_database, how='inner', left_on='Item LCA', right_on='Item') 
# Dropping columns
columns_to_drop = ['Lifetime LCA [y]', 'Item LCA', 'Item_y']
livestock_BAU_swiss = livestock_BAU_swiss.drop(columns_to_drop, axis=1)
# Renaming columns
livestock_BAU_swiss.rename(columns = {'Item_x':'Item'}, inplace = True)
# Computing the environmental impacts
target_column_index = 2  # Index of the target column to multiply other columns with
columns_to_multiply = range(3, 12)  # Range of column indices to multiply
target_column = livestock_BAU_swiss.iloc[:, target_column_index]
livestock_BAU_swiss.iloc[:, columns_to_multiply] = livestock_BAU_swiss.iloc[:, columns_to_multiply].multiply(target_column, axis=0)
# Diving by the lifetime
target_column_index = 1  # Index of the target column to divide other columns with => 'Lifetime [y]'
columns_to_multiply = range(3, 12)  # Range of column indices to multiply
target_column = livestock_BAU_swiss.iloc[:, target_column_index]
livestock_BAU_swiss.iloc[:, columns_to_multiply] = livestock_BAU_swiss.iloc[:, columns_to_multiply].divide(target_column, axis=0)
# Dropping the lifetspan and nb of animals
columns_to_drop = ['Lifespan [yr]', 'BAU Lancet']
livestock_BAU_swiss = livestock_BAU_swiss.drop(columns_to_drop, axis=1)
# Separating the df conv vs org
BAU_org = livestock_BAU_swiss[livestock_BAU_swiss['Item'].str.endswith('Org')]
BAU_conv = livestock_BAU_swiss[livestock_BAU_swiss['Item'].str.endswith('Conv')]
# Creating copies
BAU_org = BAU_org.copy()
BAU_conv = BAU_conv.copy()
# Drop '- Org' and '- Conv'
BAU_org.loc[:, 'Item'] = BAU_org['Item'].str.replace(' - Org', '')
BAU_conv.loc[:, 'Item'] = BAU_conv['Item'].str.replace(' - Conv', '')
# Setting Item as index
BAU_org = BAU_org.set_index('Item')
BAU_conv = BAU_conv.set_index('Item')
# Summing the df
livestock_BAU_swiss = BAU_conv.add(BAU_org)

# CROP, FISH & LIVESTOCK ===================================
BAU_Lancet = pd.concat([crops_fish_BAU_swiss, livestock_BAU_swiss])

# EXPORTING TO EXCEL
BAU_Lancet.to_excel(r'Desktop/PDM/Modelling_data/Environmental impacts/scenarios_from_code_31-10-2024.xlsx', sheet_name='BAU_Lancet', index=True) 
BAU_Lancet.sum()

Climate change [ kg CO2 eq]                1.933939e+09
Acidification [ mol H+ eq]                 3.466892e+07
Eutrophication, freshwater [ kg P eq]      4.342143e+05
Eutrophication, marine [ kg N eq]          1.955152e+07
Eutrophication, terrestrial [ mol N eq]    1.497193e+08
Ecotoxicity, freshwater [ CTUe]            7.320808e+10
Land use [ Pt]                             1.900378e+11
Water use [ m3 depriv,]                    4.060169e+09
Resource use, fossils [ MJ]                1.207783e+10
dtype: float64

In [12]:
# IMPACTS COMPUTATION - OA SWISS

# Preparation CROP & FISH - FEED NOT CONSIDERED
crops_fish_BAU_swiss = crop_fish_no_feed .copy()
columns_to_drop = ['Mass org BAU Swiss [kt]', 'Mass conv BAU Swiss [kt]','Mass BAU Swiss [kt]', 'Mass org BAU Lancet [kt]', 'Mass conv BAU Lancet [kt]',
                   'Mass BAU Lancet [kt]' , 'Mass OA Lancet [kt]', 'Mass org ConvBASE [kt]', 'Mass conv ConvBASE [kt]', 'Mass ConvBASE [kt]', 'Mass OrgBASE [kt]' ]
crops_fish_BAU_swiss  = crops_fish_BAU_swiss.drop(columns_to_drop, axis=1)
# Dropping the row with Citrus, other
crops_fish_BAU_swiss = crops_fish_BAU_swiss.drop(crops_fish_BAU_swiss[crops_fish_BAU_swiss['Item'] == 'Citrus, Other'].index)

# Preparation LIVESTOCK
livestock_BAU_swiss = livestock.copy()
columns_to_drop = ['BAU Lancet', 'BAU Swiss', 'OA Lancet', 'ConvBASE', 'OrgBASE']
livestock_BAU_swiss  = livestock_BAU_swiss.drop(columns_to_drop, axis=1)

# CROP & FISH ========================================
# CONVENTIONNAL AGR ___________________________
# Merging the items with conv_ch
#crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss.merge(conv_ch, on='Item', how='left') 
# Merging with the LCA database
#crops_fish_BAU_swiss_conv = pd.merge(crops_fish_BAU_swiss_conv, LCA_database, how='left', left_on='Conventionnal CH', right_on='Item')  
# Dropping columns
#columns_to_drop = ['Mass org BAU Swiss [kt]', 'Conventionnal CH', 'Item_y']
#crops_fish_BAU_swiss_conv  = crops_fish_BAU_swiss_conv.drop(columns_to_drop, axis=1)
# Renaming columns
#crops_fish_BAU_swiss_conv.rename(columns = {'Item_x':'Item'}, inplace = True)
# Computing the environmental impacts
#target_column_index = 1  # Index of the target column to multiply other columns with
#columns_to_multiply = range(2, 11)  # Range of column indices to multiply
#target_column = crops_fish_BAU_swiss_conv.iloc[:, target_column_index] * 10**6  # Converting from kt to kg
#crops_fish_BAU_swiss_conv.iloc[:, columns_to_multiply] = crops_fish_BAU_swiss_conv.iloc[:, columns_to_multiply].multiply(target_column, axis=0)

# ORGANIC AGR _________________________________
# Merging the items with org_ch
crops_fish_BAU_swiss_org = crops_fish_BAU_swiss.merge(org_ch, on='Item', how='left') 
# Merging with the LCA database
crops_fish_BAU_swiss_org = pd.merge(crops_fish_BAU_swiss_org, LCA_database, how='left', left_on='Organic CH', right_on='Item')  
# Dropping columns
columns_to_drop = ['Organic CH', 'Item_y']
crops_fish_BAU_swiss_org  = crops_fish_BAU_swiss_org.drop(columns_to_drop, axis=1)
# Renaming columns
crops_fish_BAU_swiss_org.rename(columns = {'Item_x':'Item'}, inplace = True)
# Computing the environmental impacts
target_column_index = 1  # Index of the target column to multiply other columns with
columns_to_multiply = range(2, 11)  # Range of column indices to multiply
target_column = crops_fish_BAU_swiss_org.iloc[:, target_column_index] * 10**6  # Converting from kt to kg
crops_fish_BAU_swiss_org.iloc[:, columns_to_multiply] = crops_fish_BAU_swiss_org.iloc[:, columns_to_multiply].multiply(target_column, axis=0)
# Setting Item as index
crops_fish_BAU_swiss_org = crops_fish_BAU_swiss_org.set_index('Item')
# Dropping columns
columns_to_drop = ['Mass OA Swiss [kt]']
crops_fish_BAU_swiss_org  = crops_fish_BAU_swiss_org.drop(columns_to_drop, axis=1)

# ORG & CONV _________________________________
# Filling the NaN with zeros
#crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss_conv.fillna(0)
#crops_fish_BAU_swiss_org = crops_fish_BAU_swiss_org.fillna(0)
# Setting Item as index
#crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss_conv.set_index('Item')
#crops_fish_BAU_swiss_org = crops_fish_BAU_swiss_org.set_index('Item')
#Dropping the mass columns
#columns_to_drop = ['Mass org BAU Swiss [kt]']
#crops_fish_BAU_swiss_org = crops_fish_BAU_swiss_org.drop(columns_to_drop, axis=1)
#columns_to_drop = ['Mass conv BAU Swiss [kt]']
#crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss_conv.drop(columns_to_drop, axis=1)
# Summing the df
#crops_fish_BAU_swiss = crops_fish_BAU_swiss_conv.add(crops_fish_BAU_swiss_org)

# LIVESTOCK ===================================
# Merging the items with conv_ch
livestock_BAU_swiss = livestock_BAU_swiss.merge(livestock_matched_LCA, on='Item', how='inner') 
# Merging with the LCA database
livestock_BAU_swiss = pd.merge(livestock_BAU_swiss, LCA_database, how='inner', left_on='Item LCA', right_on='Item') 
# Dropping columns
columns_to_drop = ['Lifetime LCA [y]', 'Item LCA', 'Item_y']
livestock_BAU_swiss = livestock_BAU_swiss.drop(columns_to_drop, axis=1)
# Renaming columns
livestock_BAU_swiss.rename(columns = {'Item_x':'Item'}, inplace = True)
# Computing the environmental impacts
target_column_index = 2  # Index of the target column to multiply other columns with
columns_to_multiply = range(3, 12)  # Range of column indices to multiply
target_column = livestock_BAU_swiss.iloc[:, target_column_index]
livestock_BAU_swiss.iloc[:, columns_to_multiply] = livestock_BAU_swiss.iloc[:, columns_to_multiply].multiply(target_column, axis=0)
# Diving by the lifetime
target_column_index = 1  # Index of the target column to divide other columns with => 'Lifetime [y]'
columns_to_multiply = range(3, 12)  # Range of column indices to multiply
target_column = livestock_BAU_swiss.iloc[:, target_column_index]
livestock_BAU_swiss.iloc[:, columns_to_multiply] = livestock_BAU_swiss.iloc[:, columns_to_multiply].divide(target_column, axis=0)
# Dropping the lifetspan and nb of animals
columns_to_drop = ['Lifespan [yr]', 'OA Swiss']
livestock_BAU_swiss = livestock_BAU_swiss.drop(columns_to_drop, axis=1)
# Separating the df conv vs org
BAU_org = livestock_BAU_swiss[livestock_BAU_swiss['Item'].str.endswith('Org')]
BAU_conv = livestock_BAU_swiss[livestock_BAU_swiss['Item'].str.endswith('Conv')]
# Creating copies
BAU_org = BAU_org.copy()
BAU_conv = BAU_conv.copy()
# Drop '- Org' and '- Conv'
BAU_org.loc[:, 'Item'] = BAU_org['Item'].str.replace(' - Org', '')
BAU_conv.loc[:, 'Item'] = BAU_conv['Item'].str.replace(' - Conv', '')
# Setting Item as index
BAU_org = BAU_org.set_index('Item')
BAU_conv = BAU_conv.set_index('Item')
# Summing the df
livestock_BAU_swiss = BAU_conv.add(BAU_org)

# CROP, FISH & LIVESTOCK ===================================
OA_swiss = pd.concat([crops_fish_BAU_swiss_org, livestock_BAU_swiss])

# EXPORTING TO EXCEL
OA_swiss.to_excel(r'Desktop/PDM/Modelling_data/Environmental impacts/scenarios_from_code_31-10-2024.xlsx', sheet_name='OA_swiss', index=True) 

OA_swiss.sum()

Climate change [ kg CO2 eq]                2.318033e+09
Acidification [ mol H+ eq]                 4.860211e+07
Eutrophication, freshwater [ kg P eq]      4.316083e+05
Eutrophication, marine [ kg N eq]          2.676034e+07
Eutrophication, terrestrial [ mol N eq]    2.153654e+08
Ecotoxicity, freshwater [ CTUe]            8.244840e+10
Land use [ Pt]                             2.674426e+11
Water use [ m3 depriv,]                    4.189181e+09
Resource use, fossils [ MJ]                1.041706e+10
dtype: float64

In [13]:
# IMPACTS COMPUTATION - OA Lancet

# Preparation CROP & FISH - FEED NOT CONSIDERED
crops_fish_BAU_swiss = crop_fish_no_feed .copy()
columns_to_drop = ['Mass org BAU Swiss [kt]', 'Mass conv BAU Swiss [kt]','Mass BAU Swiss [kt]', 'Mass org BAU Lancet [kt]', 'Mass conv BAU Lancet [kt]',
                   'Mass BAU Lancet [kt]' , 'Mass OA Swiss [kt]', 'Mass org ConvBASE [kt]', 'Mass conv ConvBASE [kt]', 'Mass ConvBASE [kt]', 'Mass OrgBASE [kt]' ]
crops_fish_BAU_swiss  = crops_fish_BAU_swiss.drop(columns_to_drop, axis=1)
# Dropping the row with Citrus, other
crops_fish_BAU_swiss = crops_fish_BAU_swiss.drop(crops_fish_BAU_swiss[crops_fish_BAU_swiss['Item'] == 'Citrus, Other'].index)

# Preparation LIVESTOCK
livestock_BAU_swiss = livestock.copy()
columns_to_drop = ['BAU Lancet', 'BAU Swiss', 'OA Swiss', 'ConvBASE', 'OrgBASE']
livestock_BAU_swiss  = livestock_BAU_swiss.drop(columns_to_drop, axis=1)

# CROP & FISH ========================================
# CONVENTIONNAL AGR ___________________________
# Merging the items with conv_ch
#crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss.merge(conv_ch, on='Item', how='left') 
# Merging with the LCA database
#crops_fish_BAU_swiss_conv = pd.merge(crops_fish_BAU_swiss_conv, LCA_database, how='left', left_on='Conventionnal CH', right_on='Item')  
# Dropping columns
#columns_to_drop = ['Mass org BAU Swiss [kt]', 'Conventionnal CH', 'Item_y']
#crops_fish_BAU_swiss_conv  = crops_fish_BAU_swiss_conv.drop(columns_to_drop, axis=1)
# Renaming columns
#crops_fish_BAU_swiss_conv.rename(columns = {'Item_x':'Item'}, inplace = True)
# Computing the environmental impacts
#target_column_index = 1  # Index of the target column to multiply other columns with
#columns_to_multiply = range(2, 11)  # Range of column indices to multiply
#target_column = crops_fish_BAU_swiss_conv.iloc[:, target_column_index] * 10**6  # Converting from kt to kg
#crops_fish_BAU_swiss_conv.iloc[:, columns_to_multiply] = crops_fish_BAU_swiss_conv.iloc[:, columns_to_multiply].multiply(target_column, axis=0)

# ORGANIC AGR _________________________________
# Merging the items with org_ch
crops_fish_BAU_swiss_org = crops_fish_BAU_swiss.merge(org_ch, on='Item', how='left') 
# Merging with the LCA database
crops_fish_BAU_swiss_org = pd.merge(crops_fish_BAU_swiss_org, LCA_database, how='left', left_on='Organic CH', right_on='Item')  
# Dropping columns
columns_to_drop = ['Organic CH', 'Item_y']
crops_fish_BAU_swiss_org  = crops_fish_BAU_swiss_org.drop(columns_to_drop, axis=1)
# Renaming columns
crops_fish_BAU_swiss_org.rename(columns = {'Item_x':'Item'}, inplace = True)
# Computing the environmental impacts
target_column_index = 1  # Index of the target column to multiply other columns with
columns_to_multiply = range(2, 11)  # Range of column indices to multiply
target_column = crops_fish_BAU_swiss_org.iloc[:, target_column_index] * 10**6  # Converting from kt to kg
crops_fish_BAU_swiss_org.iloc[:, columns_to_multiply] = crops_fish_BAU_swiss_org.iloc[:, columns_to_multiply].multiply(target_column, axis=0)
# Setting Item as index
crops_fish_BAU_swiss_org = crops_fish_BAU_swiss_org.set_index('Item')
# Dropping columns
columns_to_drop = ['Mass OA Lancet [kt]']
crops_fish_BAU_swiss_org  = crops_fish_BAU_swiss_org.drop(columns_to_drop, axis=1)

# ORG & CONV _________________________________
# Filling the NaN with zeros
#crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss_conv.fillna(0)
#crops_fish_BAU_swiss_org = crops_fish_BAU_swiss_org.fillna(0)
# Setting Item as index
#crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss_conv.set_index('Item')
#crops_fish_BAU_swiss_org = crops_fish_BAU_swiss_org.set_index('Item')
#Dropping the mass columns
#columns_to_drop = ['Mass org BAU Swiss [kt]']
#crops_fish_BAU_swiss_org = crops_fish_BAU_swiss_org.drop(columns_to_drop, axis=1)
#columns_to_drop = ['Mass conv BAU Swiss [kt]']
#crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss_conv.drop(columns_to_drop, axis=1)
# Summing the df
#crops_fish_BAU_swiss = crops_fish_BAU_swiss_conv.add(crops_fish_BAU_swiss_org)

# LIVESTOCK ===================================
# Merging the items with conv_ch
livestock_BAU_swiss = livestock_BAU_swiss.merge(livestock_matched_LCA, on='Item', how='inner') 
# Merging with the LCA database
livestock_BAU_swiss = pd.merge(livestock_BAU_swiss, LCA_database, how='inner', left_on='Item LCA', right_on='Item') 
# Dropping columns
columns_to_drop = ['Lifetime LCA [y]', 'Item LCA', 'Item_y']
livestock_BAU_swiss = livestock_BAU_swiss.drop(columns_to_drop, axis=1)
# Renaming columns
livestock_BAU_swiss.rename(columns = {'Item_x':'Item'}, inplace = True)
# Computing the environmental impacts
target_column_index = 2  # Index of the target column to multiply other columns with
columns_to_multiply = range(3, 12)  # Range of column indices to multiply
target_column = livestock_BAU_swiss.iloc[:, target_column_index]
livestock_BAU_swiss.iloc[:, columns_to_multiply] = livestock_BAU_swiss.iloc[:, columns_to_multiply].multiply(target_column, axis=0)
# Diving by the lifetime
target_column_index = 1  # Index of the target column to divide other columns with => 'Lifetime [y]'
columns_to_multiply = range(3, 12)  # Range of column indices to multiply
target_column = livestock_BAU_swiss.iloc[:, target_column_index]
livestock_BAU_swiss.iloc[:, columns_to_multiply] = livestock_BAU_swiss.iloc[:, columns_to_multiply].divide(target_column, axis=0)
# Dropping the lifetspan and nb of animals
columns_to_drop = ['Lifespan [yr]', 'OA Lancet']
livestock_BAU_swiss = livestock_BAU_swiss.drop(columns_to_drop, axis=1)
# Separating the df conv vs org
BAU_org = livestock_BAU_swiss[livestock_BAU_swiss['Item'].str.endswith('Org')]
BAU_conv = livestock_BAU_swiss[livestock_BAU_swiss['Item'].str.endswith('Conv')]
# Creating copies
BAU_org = BAU_org.copy()
BAU_conv = BAU_conv.copy()
# Drop '- Org' and '- Conv'
BAU_org.loc[:, 'Item'] = BAU_org['Item'].str.replace(' - Org', '')
BAU_conv.loc[:, 'Item'] = BAU_conv['Item'].str.replace(' - Conv', '')
# Setting Item as index
BAU_org = BAU_org.set_index('Item')
BAU_conv = BAU_conv.set_index('Item')
# Summing the df
livestock_BAU_swiss = BAU_conv.add(BAU_org)

# CROP, FISH & LIVESTOCK ===================================
OA_Lancet = pd.concat([crops_fish_BAU_swiss_org, livestock_BAU_swiss])

# EXPORTING TO EXCEL
OA_Lancet.to_excel(r'Desktop/PDM/Modelling_data/Environmental impacts/scenarios_from_code_31-10-2024.xlsx', sheet_name='OA_Lancet', index=True)
OA_Lancet.sum()

Climate change [ kg CO2 eq]                2.012001e+09
Acidification [ mol H+ eq]                 5.274053e+07
Eutrophication, freshwater [ kg P eq]      5.064851e+05
Eutrophication, marine [ kg N eq]          3.108901e+07
Eutrophication, terrestrial [ mol N eq]    2.328811e+08
Ecotoxicity, freshwater [ CTUe]            5.838608e+10
Land use [ Pt]                             2.754501e+11
Water use [ m3 depriv,]                    4.455869e+09
Resource use, fossils [ MJ]                1.238349e+10
dtype: float64

In [14]:
# IMPACTS COMPUTATION - ConvBASE

# Preparation CROP & FISH
crops_fish_BAU_swiss = crop_fish_no_feed.copy()
columns_to_drop = ['Mass BAU Swiss [kt]', 'Mass org BAU Swiss [kt]', 'Mass conv BAU Swiss [kt]', 'Mass OA Swiss [kt]', 'Mass OA Lancet [kt]', 'Mass BAU Lancet [kt]',
                  'Mass org BAU Lancet [kt]', 'Mass conv BAU Lancet [kt]', 'Mass ConvBASE [kt]', 'Mass OrgBASE [kt]']
crops_fish_BAU_swiss  = crops_fish_BAU_swiss.drop(columns_to_drop, axis=1)
# Dropping the row with Citrus, other
crops_fish_BAU_swiss = crops_fish_BAU_swiss.drop(crops_fish_BAU_swiss[crops_fish_BAU_swiss['Item'] == 'Citrus, Other'].index)

# Preparation LIVESTOCK
livestock_BAU_swiss = livestock.copy()
columns_to_drop = ['BAU Swiss', 'OA Lancet', 'OA Swiss', 'BAU Lancet', 'OrgBASE']
livestock_BAU_swiss  = livestock_BAU_swiss.drop(columns_to_drop, axis=1)

# CROP & FISH ========================================
# CONVENTIONNAL AGR ___________________________
# Merging the items with conv_ch
crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss.merge(conv_ch, on='Item', how='left') 
# Merging with the LCA database
crops_fish_BAU_swiss_conv = pd.merge(crops_fish_BAU_swiss_conv, LCA_database, how='left', left_on='Conventionnal CH', right_on='Item')  
# Dropping columns
columns_to_drop = ['Mass org ConvBASE [kt]', 'Conventionnal CH', 'Item_y']
crops_fish_BAU_swiss_conv  = crops_fish_BAU_swiss_conv.drop(columns_to_drop, axis=1)
# Renaming columns
crops_fish_BAU_swiss_conv.rename(columns = {'Item_x':'Item'}, inplace = True)
# Computing the environmental impacts
target_column_index = 1  # Index of the target column to multiply other columns with
columns_to_multiply = range(2, 11)  # Range of column indices to multiply
target_column = crops_fish_BAU_swiss_conv.iloc[:, target_column_index] * 10**6  # Converting from kt to kg
crops_fish_BAU_swiss_conv.iloc[:, columns_to_multiply] = crops_fish_BAU_swiss_conv.iloc[:, columns_to_multiply].multiply(target_column, axis=0)

# ORGANIC AGR _________________________________
# Merging the items with conv_ch
crops_fish_BAU_swiss_org = crops_fish_BAU_swiss.merge(org_ch, on='Item', how='left') 
# Merging with the LCA database
crops_fish_BAU_swiss_org = pd.merge(crops_fish_BAU_swiss_org, LCA_database, how='left', left_on='Organic CH', right_on='Item')  
# Dropping columns
columns_to_drop = ['Mass conv ConvBASE [kt]', 'Organic CH', 'Item_y']
crops_fish_BAU_swiss_org  = crops_fish_BAU_swiss_org.drop(columns_to_drop, axis=1)
# Renaming columns
crops_fish_BAU_swiss_org.rename(columns = {'Item_x':'Item'}, inplace = True)
# Computing the environmental impacts
target_column_index = 1  # Index of the target column to multiply other columns with
columns_to_multiply = range(2, 11)  # Range of column indices to multiply
target_column = crops_fish_BAU_swiss_org.iloc[:, target_column_index] * 10**6  # Converting from kt to kg
crops_fish_BAU_swiss_org.iloc[:, columns_to_multiply] = crops_fish_BAU_swiss_org.iloc[:, columns_to_multiply].multiply(target_column, axis=0)

# ORG & CONV _________________________________
# Filling the NaN with zeros
crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss_conv.fillna(0)
crops_fish_BAU_swiss_org = crops_fish_BAU_swiss_org.fillna(0)
# Setting Item as index
crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss_conv.set_index('Item')
crops_fish_BAU_swiss_org = crops_fish_BAU_swiss_org.set_index('Item')
#Dropping the mass columns
columns_to_drop = ['Mass org ConvBASE [kt]']
crops_fish_BAU_swiss_org = crops_fish_BAU_swiss_org.drop(columns_to_drop, axis=1)
columns_to_drop = ['Mass conv ConvBASE [kt]']
crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss_conv.drop(columns_to_drop, axis=1)
# Summing the df
crops_fish_BAU_swiss = crops_fish_BAU_swiss_conv.add(crops_fish_BAU_swiss_org)

# LIVESTOCK ===================================
# Merging the items with conv_ch
livestock_BAU_swiss = livestock_BAU_swiss.merge(livestock_matched_LCA, on='Item', how='inner') 
# Merging with the LCA database
livestock_BAU_swiss = pd.merge(livestock_BAU_swiss, LCA_database, how='inner', left_on='Item LCA', right_on='Item') 
# Dropping columns
columns_to_drop = ['Lifetime LCA [y]', 'Item LCA', 'Item_y']
livestock_BAU_swiss = livestock_BAU_swiss.drop(columns_to_drop, axis=1)
# Renaming columns
livestock_BAU_swiss.rename(columns = {'Item_x':'Item'}, inplace = True)
# Computing the environmental impacts
target_column_index = 2  # Index of the target column to multiply other columns with
columns_to_multiply = range(3, 12)  # Range of column indices to multiply
target_column = livestock_BAU_swiss.iloc[:, target_column_index]
livestock_BAU_swiss.iloc[:, columns_to_multiply] = livestock_BAU_swiss.iloc[:, columns_to_multiply].multiply(target_column, axis=0)
# Diving by the lifetime
target_column_index = 1  # Index of the target column to divide other columns with => 'Lifetime [y]'
columns_to_multiply = range(3, 12)  # Range of column indices to multiply
target_column = livestock_BAU_swiss.iloc[:, target_column_index]
livestock_BAU_swiss.iloc[:, columns_to_multiply] = livestock_BAU_swiss.iloc[:, columns_to_multiply].divide(target_column, axis=0)
# Dropping the lifetspan and nb of animals
columns_to_drop = ['Lifespan [yr]', 'ConvBASE']
livestock_BAU_swiss = livestock_BAU_swiss.drop(columns_to_drop, axis=1)
# Separating the df conv vs org
BAU_org = livestock_BAU_swiss[livestock_BAU_swiss['Item'].str.endswith('Org')]
BAU_conv = livestock_BAU_swiss[livestock_BAU_swiss['Item'].str.endswith('Conv')]
# Creating copies
BAU_org = BAU_org.copy()
BAU_conv = BAU_conv.copy()
# Drop '- Org' and '- Conv'
BAU_org.loc[:, 'Item'] = BAU_org['Item'].str.replace(' - Org', '')
BAU_conv.loc[:, 'Item'] = BAU_conv['Item'].str.replace(' - Conv', '')
# Setting Item as index
BAU_org = BAU_org.set_index('Item')
BAU_conv = BAU_conv.set_index('Item')
# Summing the df
livestock_BAU_swiss = BAU_conv.add(BAU_org)

# CROP, FISH & LIVESTOCK ===================================
BAU_Lancet = pd.concat([crops_fish_BAU_swiss, livestock_BAU_swiss])

# EXPORTING TO EXCEL
BAU_Lancet.to_excel(r'Desktop/PDM/Modelling_data/Environmental impacts/scenarios_from_code_31-10-2024.xlsx', sheet_name='ConvBASE', index=True) 
BAU_Lancet.sum()

Climate change [ kg CO2 eq]                5.462577e+09
Acidification [ mol H+ eq]                 9.117467e+07
Eutrophication, freshwater [ kg P eq]      9.043298e+05
Eutrophication, marine [ kg N eq]          4.034169e+07
Eutrophication, terrestrial [ mol N eq]    3.989694e+08
Ecotoxicity, freshwater [ CTUe]            1.607343e+11
Land use [ Pt]                             5.031725e+11
Water use [ m3 depriv,]                    5.882473e+09
Resource use, fossils [ MJ]                2.534673e+10
dtype: float64

In [10]:
# IMPACTS COMPUTATION - OrgBASE

# Preparation CROP & FISH - FEED NOT CONSIDERED
crops_fish_BAU_swiss = crop_fish_no_feed .copy()
columns_to_drop = ['Mass org BAU Swiss [kt]', 'Mass conv BAU Swiss [kt]','Mass BAU Swiss [kt]', 'Mass org BAU Lancet [kt]', 'Mass conv BAU Lancet [kt]',
                   'Mass BAU Lancet [kt]' , 'Mass OA Swiss [kt]', 'Mass OA Lancet [kt]', 'Mass org ConvBASE [kt]', 'Mass conv ConvBASE [kt]', 'Mass ConvBASE [kt]' ]
crops_fish_BAU_swiss  = crops_fish_BAU_swiss.drop(columns_to_drop, axis=1)
# Dropping the row with Citrus, other
crops_fish_BAU_swiss = crops_fish_BAU_swiss.drop(crops_fish_BAU_swiss[crops_fish_BAU_swiss['Item'] == 'Citrus, Other'].index)

# Preparation LIVESTOCK
livestock_BAU_swiss = livestock.copy()
columns_to_drop = ['BAU Lancet', 'BAU Swiss', 'OA Swiss', 'ConvBASE', 'OA Lancet']
livestock_BAU_swiss  = livestock_BAU_swiss.drop(columns_to_drop, axis=1)

# CROP & FISH ========================================
# CONVENTIONNAL AGR ___________________________
# Merging the items with conv_ch
#crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss.merge(conv_ch, on='Item', how='left') 
# Merging with the LCA database
#crops_fish_BAU_swiss_conv = pd.merge(crops_fish_BAU_swiss_conv, LCA_database, how='left', left_on='Conventionnal CH', right_on='Item')  
# Dropping columns
#columns_to_drop = ['Mass org BAU Swiss [kt]', 'Conventionnal CH', 'Item_y']
#crops_fish_BAU_swiss_conv  = crops_fish_BAU_swiss_conv.drop(columns_to_drop, axis=1)
# Renaming columns
#crops_fish_BAU_swiss_conv.rename(columns = {'Item_x':'Item'}, inplace = True)
# Computing the environmental impacts
#target_column_index = 1  # Index of the target column to multiply other columns with
#columns_to_multiply = range(2, 11)  # Range of column indices to multiply
#target_column = crops_fish_BAU_swiss_conv.iloc[:, target_column_index] * 10**6  # Converting from kt to kg
#crops_fish_BAU_swiss_conv.iloc[:, columns_to_multiply] = crops_fish_BAU_swiss_conv.iloc[:, columns_to_multiply].multiply(target_column, axis=0)

# ORGANIC AGR _________________________________
# Merging the items with org_ch
crops_fish_BAU_swiss_org = crops_fish_BAU_swiss.merge(org_ch, on='Item', how='left') 
# Merging with the LCA database
crops_fish_BAU_swiss_org = pd.merge(crops_fish_BAU_swiss_org, LCA_database, how='left', left_on='Organic CH', right_on='Item')  
# Dropping columns
columns_to_drop = ['Organic CH', 'Item_y']
crops_fish_BAU_swiss_org  = crops_fish_BAU_swiss_org.drop(columns_to_drop, axis=1)
# Renaming columns
crops_fish_BAU_swiss_org.rename(columns = {'Item_x':'Item'}, inplace = True)
# Computing the environmental impacts
target_column_index = 1  # Index of the target column to multiply other columns with
columns_to_multiply = range(2, 11)  # Range of column indices to multiply
target_column = crops_fish_BAU_swiss_org.iloc[:, target_column_index] * 10**6  # Converting from kt to kg
crops_fish_BAU_swiss_org.iloc[:, columns_to_multiply] = crops_fish_BAU_swiss_org.iloc[:, columns_to_multiply].multiply(target_column, axis=0)
# Setting Item as index
crops_fish_BAU_swiss_org = crops_fish_BAU_swiss_org.set_index('Item')
# Dropping columns
columns_to_drop = ['Mass OrgBASE [kt]']
crops_fish_BAU_swiss_org  = crops_fish_BAU_swiss_org.drop(columns_to_drop, axis=1)

# ORG & CONV _________________________________
# Filling the NaN with zeros
#crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss_conv.fillna(0)
#crops_fish_BAU_swiss_org = crops_fish_BAU_swiss_org.fillna(0)
# Setting Item as index
#crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss_conv.set_index('Item')
#crops_fish_BAU_swiss_org = crops_fish_BAU_swiss_org.set_index('Item')
#Dropping the mass columns
#columns_to_drop = ['Mass org BAU Swiss [kt]']
#crops_fish_BAU_swiss_org = crops_fish_BAU_swiss_org.drop(columns_to_drop, axis=1)
#columns_to_drop = ['Mass conv BAU Swiss [kt]']
#crops_fish_BAU_swiss_conv = crops_fish_BAU_swiss_conv.drop(columns_to_drop, axis=1)
# Summing the df
#crops_fish_BAU_swiss = crops_fish_BAU_swiss_conv.add(crops_fish_BAU_swiss_org)

# LIVESTOCK ===================================
# Merging the items with conv_ch
livestock_BAU_swiss = livestock_BAU_swiss.merge(livestock_matched_LCA, on='Item', how='inner') 
# Merging with the LCA database
livestock_BAU_swiss = pd.merge(livestock_BAU_swiss, LCA_database, how='inner', left_on='Item LCA', right_on='Item') 
# Dropping columns
columns_to_drop = ['Lifetime LCA [y]', 'Item LCA', 'Item_y']
livestock_BAU_swiss = livestock_BAU_swiss.drop(columns_to_drop, axis=1)
# Renaming columns
livestock_BAU_swiss.rename(columns = {'Item_x':'Item'}, inplace = True)
# Computing the environmental impacts
target_column_index = 2  # Index of the target column to multiply other columns with
columns_to_multiply = range(3, 12)  # Range of column indices to multiply
target_column = livestock_BAU_swiss.iloc[:, target_column_index]
livestock_BAU_swiss.iloc[:, columns_to_multiply] = livestock_BAU_swiss.iloc[:, columns_to_multiply].multiply(target_column, axis=0)
# Diving by the lifetime
target_column_index = 1  # Index of the target column to divide other columns with => 'Lifetime [y]'
columns_to_multiply = range(3, 12)  # Range of column indices to multiply
target_column = livestock_BAU_swiss.iloc[:, target_column_index]
livestock_BAU_swiss.iloc[:, columns_to_multiply] = livestock_BAU_swiss.iloc[:, columns_to_multiply].divide(target_column, axis=0)
# Dropping the lifetspan and nb of animals
columns_to_drop = ['Lifespan [yr]', 'OrgBASE']
livestock_BAU_swiss = livestock_BAU_swiss.drop(columns_to_drop, axis=1)
# Separating the df conv vs org
BAU_org = livestock_BAU_swiss[livestock_BAU_swiss['Item'].str.endswith('Org')]
BAU_conv = livestock_BAU_swiss[livestock_BAU_swiss['Item'].str.endswith('Conv')]
# Creating copies
BAU_org = BAU_org.copy()
BAU_conv = BAU_conv.copy()
# Drop '- Org' and '- Conv'
BAU_org.loc[:, 'Item'] = BAU_org['Item'].str.replace(' - Org', '')
BAU_conv.loc[:, 'Item'] = BAU_conv['Item'].str.replace(' - Conv', '')
# Setting Item as index
BAU_org = BAU_org.set_index('Item')
BAU_conv = BAU_conv.set_index('Item')
# Summing the df
livestock_BAU_swiss = BAU_conv.add(BAU_org)

# CROP, FISH & LIVESTOCK ===================================
OA_Lancet = pd.concat([crops_fish_BAU_swiss_org, livestock_BAU_swiss])

# EXPORTING TO EXCEL
OA_Lancet.to_excel(r'Desktop/PDM/Modelling_data/Environmental impacts/scenarios_from_code_31-10-2024.xlsx', sheet_name='OrgBASE', index=True)
#OA_Lancet